In [2]:

import transformers
import json

import datasets
squad_dataset = datasets.load_dataset('squad')
from datasets import concatenate_datasets, load_dataset

covid_file = '../data/COVID-QA.json'
bio_file = '../bioASQ/bioASQ.json'

def make_and_save_full_dataset(train, valid, test, path):
    full_data = datasets.dataset_dict.DatasetDict({'train':train, 'validation':valid, 'test': test})
    full_data.save_to_disk(path)

def get_dataset(filename):
    return datasets.load_dataset('custom_squad.py', data_files= {'train':filename})['train']

Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\plain_text\1.0.0\6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16)


In [3]:
import run_qa

def run_gradual_ft(output_dir, checkpoint, covid_val):
    !python run_qa.py \
      --model_name_or_path {checkpoint} \
      --dataset_name ../data/full_squad_covidQA/ \
      --do_train \
      --do_eval \
      --do_predict \
      --per_device_train_batch_size 32\
      --per_device_eval_batch_size 32\
      --evaluation_strategy "no" \
      --save_strategy "no" \
      --logging_strategy "epoch" \
      --learning_rate 1e-5 \
      --num_train_epochs 1 \
      --max_seq_length 384 \
      --doc_stride 128 \
      --output_dir {output_dir} \
      --overwrite_output_dir

In [4]:
data_files = {}
data_files["train"] = covid_file

covid_qa = get_dataset(covid_file)
bio_qa = get_dataset(bio_file)

squad_qa = concatenate_datasets([squad_dataset['train'], squad_dataset['validation']])
covid_and_squad_dataset_path = "../data/full_squad_covidQA"

squad_qa = datasets.Dataset.from_dict(squad_qa[:50])
covid_qa = datasets.Dataset.from_dict(covid_qa[:20])
bio_qa = datasets.Dataset.from_dict(bio_qa[:20])

Using custom data configuration default-a396986d6b3a2375
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-a396986d6b3a2375\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)
Using custom data configuration default-8fdbe041288a2f4d
Reusing dataset squad (C:\Users\unitu\.cache\huggingface\datasets\squad\default-8fdbe041288a2f4d\0.0.0\cb00e306c4924563ce3d1292a1ce1b86b2753dab6285ce43c87b39c5bda3ef4e)


In [26]:
k_fold = 5
K = 6

for i in range(k_fold):
    covid_fold = covid_qa.shard(k_fold, i)

    covid_test = covid_fold.shard(2, 0)
    covid_val = covid_fold.shard(2, 1)
    covid_train = concatenate_datasets([covid_qa.shard(k_fold, j) for j in range(k_fold) if j != i])

    #make_and_save_full_dataset(covid_train, squad_qa, covid_val, covid_test, covid_and_squad_dataset_path)

    checkpoint = 'roberta-base'
    cur_dir = '../models/gradual_ft_baseline_lr1e-5_2/split_' + str(i)

    squad_qa.shuffle()
    bio_qa.shuffle()

    num_of_shards = int(K/2)
    squad_qa_shards = [squad_qa.shard(num_of_shards,i) for i in range(num_of_shards)]
    bio_qa_shards = [bio_qa.shard(num_of_shards,i) for i in range(num_of_shards)]
    for n in range(K):
        output_dir = cur_dir + '/checkpoint_' + str(n)
        try:
            if n%2:
                squad_qa_cur = concatenate_datasets(squad_qa_shards)
                full_dataset = concatenate_datasets([squad_qa_cur, covid_train])
                print('squad: ', squad_qa_cur.num_rows, 'n = ',n)
                _ = squad_qa_shards.pop()
            else:
                bio_qa_cur = concatenate_datasets(bio_qa_shards)
                full_dataset = concatenate_datasets([bio_qa_cur, covid_train])
                print('bio: ', bio_qa_cur.num_rows, 'n= ', n)
            _ = bio_qa_shards.pop()
        except:
            full_dataset = covid_train

        make_and_save_full_dataset(full_dataset, covid_val, covid_test, covid_and_squad_dataset_path)

        run_gradual_ft(output_dir, checkpoint, covid_val)

        checkpoint = output_dir

06/30/2021 20:52:17 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/30/2021 20:52:17 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|configuration_utils.py:530] 2021-06-30 20:52:17,865 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\unitu/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:566] 2021-06-30 20:52:17,866 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "u

06/30/2021 20:52:38 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/30/2021 20:52:38 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|configuration_utils.py:528] 2021-06-30 20:52:38,888 >> loading configuration file ../models/gradual_ft_baseline_lr1e-5_2/split_0/checkpoint_0\config.json
[INFO|configuration_utils.py:566] 2021-06-30 20:52:38,888 >> Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_utils_base.py:1651] 2021-06-30 20:52:38,896 >> Didn't find file ../models/g

06/30/2021 20:52:55 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/30/2021 20:52:55 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

...

[INFO|configuration_utils.py:528] 2021-06-30 21:03:18,072 >> loading configuration file ../models/gradual_ft_baseline_lr1e-5_2/split_4/checkpoint_1\config.json
[INFO|configuration_utils.py:566] 2021-06-30 21:03:18,073 >> Model config RobertaConfig {
  "_name_or_path": "../models/gradual_ft_baseline_lr1e-5_2/split_4/checkpoint_0",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_utils_base.py:1651] 2021-06-

06/30/2021 21:03:33 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/30/2021 21:03:33 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=500,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=None,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=False,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=../models

[INFO|configuration_utils.py:528] 2021-06-30 21:03:33,892 >> loading configuration file ../models/gradual_ft_baseline_lr1e-5_2/split_4/checkpoint_2\config.json
[INFO|configuration_utils.py:566] 2021-06-30 21:03:33,893 >> Model config RobertaConfig {
  "_name_or_path": "../models/gradual_ft_baseline_lr1e-5_2/split_4/checkpoint_1",
  "architectures": [
    "RobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

[INFO|tokenization_utils_base.py:1651] 2021-06-

    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 2
    })
    test: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 2
    })
})
11111111111111111111111111111111111111111111111111111111111111111111
88888888888888888888888888888888888888888888888888888888888888888888
                 BioASQ data not included
88888888888888888888888888888888888888888888888888888888888888888888
11111111111111111111111111111111111111111111111111111111111111111111
***************************
Split  1
***************************
model path ====  ../models/gradual_ft_baseline_lr1e-5_2/split_4/checkpoint_2
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Datasets DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 16
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'titl

[INFO|trainer.py:2146] 2021-06-30 21:03:46,481 >> ***** Running Prediction *****
[INFO|trainer.py:2148] 2021-06-30 21:03:46,481 >>   Num examples = 76
[INFO|trainer.py:2151] 2021-06-30 21:03:46,481 >>   Batch size = 32

 67%|######6   | 2/3 [00:00<00:00, 14.71it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

100%|##########| 2/2 [00:00<00:00, 10.10it/s]

100%|##########| 3/3 [00:00<00:00,  5.65it/s]


in n < K-1 40
in n < K-1 30
in n < K-1 20
in n < K-1 10
10
